In [3]:
# Printing all outputs in a cell (not only the last output)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## ** 머신러닝 3주차 실습 (Due 6/15 10pm) **

### Deep Neural Networks

---
#### ** 1) [심층 신경망의 구성과 자동화] ** 
   
* (입력층: 노드수), (은닉층: 은닉층의 수, 각 은닉층의 노드수, 각 은닉층의 활성화 함수 종류), (출력층: 노드수, 출력층의 활성화 함수 종류)를
  기본적인 신경망 구성의 파라메터로 받아서, 복수의 은닉층을 가진 신경망을 편리하게 구성해주는 클래스를 작성해보자.  
  
* 이때 각 층에서 선택 가능한 활성화 함수로써, 은닉층에는 (sigmoid, tanh, ReLU), 출력층에는 (identity, sigmoid, softmax)의 
  자유로운 선택이 가능하도록 신경망 클래스를 확장해보고, 층층이 다른 활성화 함수를 사용할 경우 이에 걸맞는 역전파가 가능하도록 train 함수를 확장해보자. 

* 더 나아가, 일반적인 활성화 함수의 사용이 가능하도록, 수치미분함수를 도입하여 역전파 및 train 함수를 작성해보자.

* 가중치의 초기값분포에 따라서 각 층에서의 출력분포가 어떻게 달라지는지 확인하여보자.

* 학습한 모형의 파라메터들을 저장하고, 언제든지 다시 로드하여 훈련된 신경망 모형을 활용할 수 있도록 구성해보자.


In [4]:
# 데이터 불러오기 (MNIST 손글씨 데이터)

# 훈련데이터
training_data_file = open("mnist_train.csv",'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

# 테스트데이터
test_data_file = open("mnist_test.csv",'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [4]:
len(training_data_list)
len(test_data_list)

60000

10000

In [17]:
%reload_ext autoreload
%autoreload 2

# NNfactory.py 임포트
import NNfactory

In [18]:
name = 'myMLP'
structure = '784:identity|100:tanh|100:tanh|10:softmax'
mynn = NNfactory.MLP(model_structure = structure, model_nametag = name, learning_rate=0.005)


 * 다음과 같은 구조의 다층퍼셉트론 연결이 초기화 되었습니다 *

 > 모델이름 = myMLP
 > 총 층수 (입력 + 은닉(s) + 출력) =  4
 > 각 층에서의 노드수 =  [784, 100, 100, 10]
 > 각 층에서의 활성화 함수 =  ['identity', 'tanh', 'tanh', 'softmax']
 > 학습률(Learning Rate) =  0.005


In [5]:
from time import sleep
import sys
import numpy as np

# 네트웍의 훈련: the neural network
data_type = 'mnist'

# 최대 학습 주기 설정
epochs = 100

n_data_tot=len(training_data_list)
n_data_max=60000  # 훈련에 사용할 데이터 갯수 (최대 60000)
n_data_test=min(int((n_data_tot-n_data_max)*0.7),int(n_data_max*0.7))
n_data = len(training_data_list[:n_data_max])
dn_data = int(n_data/20)
print (dn_data)

for e in range(epochs):
    # go through all records in the training data set
    print(' * epoch = {}'.format(e+1))
    id_data = 0
    
    for record in training_data_list[:n_data_max]:
        
        id_data += 1
        if (id_data%dn_data==0):
            sys.stdout.write('\r')
            sys.stdout.write(' [%-20s] %d%%' % ( '='*(id_data//dn_data), 5*(id_data//dn_data)))
            sys.stdout.flush()
            sleep(0.25)
        
        # split the record by the ',' commas
        all_values = record.split(',')

        # 입력데이터 ['x1','x2',...] & 입력데이터의 스케일링 (n_input_nodes, )
        if data_type == 'mnist':
            input_list = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        else:
            input_list = np.asfarray(all_values[1:])

    
        # create the target output values (shape = (10,))
        target_list = np.zeros(10) #mynn.n_nodes[-1])
        
        # all_values[0] is the target label for this record
        target_list[int(all_values[0])] = 1.0

        mynn.train(input_list, target_list)
        
        pass
    
    print('')
    print(' > 훈련 샘플에 대한 성능 (정확도 & 평균에러) ')
    mynn.check_accuracy_error(training_data_list, 0, n_data_max, data_type='mnist')
    print('')
    print(' > 테스트 샘플에 대한 성능 (정확도 & 평균에러) ')
    mynn.check_accuracy_error(test_data_list, 0, len(test_data_list), data_type='mnist')
        
    print('\n')
    
    pass

500
 * epoch = 1
 [====================] 100%
 > 훈련 샘플 학습도 (정확도 & 평균에러) 
 
 * 현재 정확도  =  0.894710528947  | (정답수)/(테스트 데이터수) =  8948 / 10001
 * 현재 평균에러 =  0.0160944238297

 > 테스트 샘플 학습도 (정확도 & 평균에러) 
 
 * 현재 정확도  =  0.8813  | (정답수)/(테스트 데이터수) =  8813 / 10000
 * 현재 평균에러 =  0.0182080230061


 * epoch = 2
 [================    ] 80%

KeyboardInterrupt: 

* 훈련한 모형의 저장

훈련된 모형은 NNfactory클래스 안의 save_model메소드를 사용하여 .npy포맷의 numpy array로 저장할 수 있다.

In [24]:
mynn.save_model(fname='mymodel.npy', nametag='first run')

* 저장된 모형 불러오기 :

방법1) 껍데기 신경망 인스턴스를 선언하고, 그 인스턴스의 load_model메소드를 이용하여 저장된 .npy파일로부터 신경망을 직접 로드한다 

In [25]:
mynn2 = NNfactory.MLP()


 * 다음과 같은 구조의 다층퍼셉트론 연결이 초기화 되었습니다 *

 > 모델이름 = a MLP
 > 총 층수 (입력 + 은닉(s) + 출력) =  3
 > 각 층에서의 노드수 =  [784, 100, 10]
 > 각 층에서의 활성화 함수 =  ['identity', 'tanh', 'softmax']
 > 학습률(Learning Rate) =  0.005


In [26]:
mynn2.load_model(fname='mymodel.npy')


 * 다음과 같은 구조의 다층퍼셉트론 모형이 "load_model_np" 정보로부터 로드되었습니다. *

 > 모델이름 = first run
 > 총 층수 (입력 + 은닉(s) + 출력) =  4
 > 각 층에서의 노드수 =  [784, 100, 100, 10]
 > 각 층에서의 활성화 함수 =  ['identity', 'tanh', 'tanh', 'softmax']
 > 학습률(Learning Rate) =  0.005


* 저장된 모형 불러오기 :

방법2) 저장된 .npy파일로부터 신경망정보가 담긴 넘파이 배열을 직접 로드하고, 이 넘파이 배열을 새 신경망 인스턴스 생성에 사용하여 저장된 모형과 똑같은 신경망을 로드한다.

In [27]:
mymodel = np.load('mymodel.npy')
mynn3 = NNfactory.MLP(load_model_np=mymodel)


 * 다음과 같은 구조의 다층퍼셉트론 모형이 "load_model_np" 정보로부터 로드되었습니다. *

 > 모델이름 = first run
 > 총 층수 (입력 + 은닉(s) + 출력) =  4
 > 각 층에서의 노드수 =  [784, 100, 100, 10]
 > 각 층에서의 활성화 함수 =  ['identity', 'tanh', 'tanh', 'softmax']
 > 학습률(Learning Rate) =  0.005


In [28]:
data = training_data_list[0].split(',')
data[0]
input_list = np.asfarray(data[1:])/255*0.99 + 0.01
# input_list

In [29]:
# 로드된 모형(mynn2)의 순방향 출력 체크 (첫번째 데이터에 대하여) 
np.argmax(mynn2.feedforward(input_list))

3

---
#### ** 2) [과적합(Overfititng)의 탐지 및 방지] ** 
   
* 주어진 난수 데이터에 대하여, 은닉층의 수가 늘어남에 따라서 학습 곡선이 어떻게 달라지는지 체크해보자.
   
* 머신러닝 2주차의 회귀 모형의 신경망 학습에 대하여 과적합이 일어난 상황을 만들고, 이를 확인해보자.
   
